In [106]:
%matplotlib inline

In [112]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import numpy as np
import pandas as pd
from pandas import Series
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from pipeline import Dataset
from datatools import data_quantization, extract_market_data

In [2]:
dataset = Dataset.load('../../data/parsed')

In [4]:
m_df = dataset.market.set_index(['asset','day','timeslot'])
m_df.head()

,,,open,close,high,low,volume,money
asset,day,timeslot,,,,,,
0,1,1,25.0164,24.9557,25.1378,24.9436,642243.0,1.610060e+07
1,1,1,16.3256,16.3499,16.4106,16.3256,15552.0,2.540301e+05
2,1,1,9.2006,9.1763,9.2006,9.1521,396647.0,3.638304e+06
3,1,1,9.1521,9.2006,9.2006,9.1521,317044.0,2.907092e+06
4,1,1,5.1829,5.1222,5.1829,5.1101,294784.0,1.514496e+06


In [5]:
m_df_day = extract_market_data(m_df)

/Users/lewisliu/Desktop/Study/Graduate_Study_Austin/Career/Practice/qids-2023-comp/datatools.py:216: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  m_df_day = m_df.groupby(level=[0, 1])['volume', 'money'].sum()


In [8]:
m_df_day.head()

avg_price  volatility  mean_volume
day asset                                    
1   0      24.553713    0.861989    454075.84
    1      16.169781    0.804576    176868.36
    2       9.059178    0.351893    361648.66
    3       9.124460    0.250697    351809.54
    4       5.054415    0.270981    338476.86

In [ ]:
m_df.iloc[1]['money']/m_df.iloc[1]['volume']


In [69]:
# sort indexing
m_df = m_df.swaplevel(0,1)
m_df.reset_index(inplace=True)
m_df = m_df.sort_values(['day', 'asset', 'timeslot'], ascending=[True,True,True])
# # convert back to multi_index
m_df.set_index(['day','asset','timeslot'], inplace=True)
m_df.head()


open    close     high      low     volume  \
day asset timeslot                                                  
1   0     1         25.0164  24.9557  25.1378  24.9436   642243.0   
          2         24.9557  24.8465  24.9557  24.8101   797872.0   
          3         24.8465  24.7008  24.8586  24.6644  1234546.0   
          4         24.7008  24.7615  24.7858  24.7008   533023.0   
          5         24.7615  24.7251  24.7615  24.7008   565000.0   

                           money  
day asset timeslot                
1   0     1         1.610060e+07  
          2         1.984058e+07  
          3         3.056567e+07  
          4         1.318848e+07  
          5         1.396949e+07

In [8]:
m_df_backup = m_df.copy()

In [ ]:
m_df_day.query('day==372 and asset==21')
m_df_day.loc[(372,21),'open':'low']

Try to compute some statistics

MultiIndex([(  1,  0),
            (372, 21)],
           names=['day', 'asset'])

In [70]:
m_df_day = m_df.groupby(level=[0,1]).sum()
#compute average price
m_df_day['avg_price'] = m_df_day['money']/m_df_day['volume']
# find index of zero volume
indx_day = m_df_day[m_df_day['volume']==0].index
# compute replace value as mean of high and low
for (i, indx) in enumerate(indx_day):
    replace_value = .5*m_df.loc[indx,'high'].max()+.5*m_df.loc[indx,'low'].min()
    m_df.loc[indx,'open':'low']=replace_value.item()
    m_df_day.loc[indx, 'avg_price'] = replace_value.item()

In [72]:
m_df_day.head()

open      close       high        low      volume  \
day asset                                                           
1   0      1227.5277  1226.8360  1228.9359  1225.4038  22703792.0   
    1       808.0876   807.9541   810.2361   806.1333   8843418.0   
    2       452.7961   452.5896   453.6701   451.6673  18082433.0   
    3       455.6850   455.6973   456.3893   455.1508  17590477.0   
    4       252.8829   252.6522   253.4174   252.1059  16923843.0   

                  money  avg_price  
day asset                           
1   0      5.574624e+08  24.553713  
    1      1.429961e+08  16.169781  
    2      1.638120e+08   9.059178  
    3      1.605036e+08   9.124460  
    4      8.554012e+07   5.054415

In [73]:
m_df.head()

open    close     high      low     volume  \
day asset timeslot                                                  
1   0     1         25.0164  24.9557  25.1378  24.9436   642243.0   
          2         24.9557  24.8465  24.9557  24.8101   797872.0   
          3         24.8465  24.7008  24.8586  24.6644  1234546.0   
          4         24.7008  24.7615  24.7858  24.7008   533023.0   
          5         24.7615  24.7251  24.7615  24.7008   565000.0   

                           money  
day asset timeslot                
1   0     1         1.610060e+07  
          2         1.984058e+07  
          3         3.056567e+07  
          4         1.318848e+07  
          5         1.396949e+07

In [90]:
T = 50.0 # number of time units
m_df_std = m_df.groupby(level=[0,1])['close'].std()
# note numpy use 0 dof while pd use 1 dof
m_df_day['volatility'] = m_df_std*np.sqrt(T)

In [92]:
m_df_day['volume'] = m_df_day['volume']/50

In [107]:
m_df

open    close     high      low     volume  \
day  asset timeslot                                                  
1    0     1         25.0164  24.9557  25.1378  24.9436   642243.0   
           2         24.9557  24.8465  24.9557  24.8101   797872.0   
           3         24.8465  24.7008  24.8586  24.6644  1234546.0   
           4         24.7008  24.7615  24.7858  24.7008   533023.0   
           5         24.7615  24.7251  24.7615  24.7008   565000.0   
...                      ...      ...      ...      ...        ...   
1000 53    46         9.8439   9.8803   9.8925   9.8439   160915.0   
           47         9.8803   9.9167   9.9289   9.8803   237961.0   
           48         9.9167   9.9289   9.9289   9.9167   182479.0   
           49         9.9289   9.8803   9.9289   9.8682   267976.0   
           50         9.8803   9.9046   9.9046   9.8803   139460.0   

                            money  
day  asset timeslot                
1    0     1         1.610060e+07  
           2         1.984058e+07  
           3         3.056567e+07  
           4         1.318848e+07  
           5         1.396949e+07  
...                           ...  
1000 53    46        1.587971e+06  
           47        2.356753e+06  
           48        1.809533e+06  
           49        2.653046e+06  
           50        1.380952e+06  

[2700000 rows x 6 columns]

In [94]:
m_df.groupby(level=[0,1])['volume'].mean()


day   asset
1     0        454075.84
      1        176868.36
      2        361648.66
      3        351809.54
      4        338476.86
                 ...    
1000  49        93683.78
      50        71372.44
      51       391628.82
      52       109396.98
      53       208841.56
Name: volume, Length: 54000, dtype: float64

In [95]:
np.size(m_df_day.isna().sum(axis=1).to_numpy().nonzero()[
                       0]) == 0


True

In [7]:
m_df.groupby(level=[0, 1])[['volume', 'money']].sum()

volume         money
asset day                            
0     1      22703792.0  5.574624e+08
      2      20075851.0  4.800800e+08
      3      15398462.0  3.637981e+08
      4      28162025.0  6.723506e+08
      5     112498597.0  2.939903e+09
...                 ...           ...
53    996     8227288.0  7.770097e+07
      997    14086031.0  1.387734e+08
      998    12641373.0  1.241513e+08
      999    11411585.0  1.143978e+08
      1000   10442078.0  1.037523e+08

[54000 rows x 2 columns]